In [1]:
MAX_COUNT = 25

In [2]:
import os
os.chdir("Data")
!ls

Challenge               dataset_labels_small.h5 pair_huge.json
dataset.h5              leaderboard.h5          test.h5
dataset_contrast.h5     molecules.csv           train.h5
dataset_labels.h5       odor_pair_full.json


In [3]:
import json
with open("pair_huge.json") as f:
    dataset = json.load(f)

len(dataset),dataset[0]

(266758,
 {'mol1': 'CC(=CCC1C(O1)(C)C)C=C',
  'mol2': 'CCCC(CC)O',
  'blend_notes': ['herbal']})

In [4]:
import tqdm

all_notes = set()
for d in tqdm.tqdm(dataset):
    all_notes.update(d["blend_notes"])
all_notes = list(all_notes)

all_notes

100%|████████████████████████████| 266758/266758 [00:00<00:00, 4079500.29it/s]


['clean',
 'vegetable',
 'honey',
 'fishy',
 'lactonic',
 'mentholic',
 'marine',
 'fruity',
 'milky',
 'mossy',
 'tobacco',
 'meaty',
 'tropical',
 'coffee',
 'radish',
 'floral',
 'bready',
 'kokumi',
 'green',
 'alcoholic',
 'medicinal',
 'sulfurous',
 'juicy',
 'creamy',
 'thujonic',
 'terpenic',
 'savory',
 'apple',
 'chocolate',
 'anise',
 'acrylate',
 'naphthyl',
 'sweet',
 'moldy',
 'pungent',
 'brown',
 'balsamic',
 'ammoniacal',
 'melon',
 'musty',
 'aldehydic',
 'herbal',
 'winey',
 'nutty',
 'mustard',
 'dry',
 'rummy',
 'cheesy',
 'roasted',
 'solvent',
 'malty',
 'salty',
 'orris',
 'astringent',
 'tarragon',
 'cocoa',
 'jammy',
 'vanilla',
 'seafood',
 'hay',
 'sweaty',
 'berry',
 'coumarinic',
 'pine',
 'waxy',
 'rooty',
 'animal',
 'toasted',
 'corn chip',
 'peach',
 'aromatic',
 'metallic',
 'alliaceous',
 'celery',
 'woody',
 'camphoreous',
 'estery',
 'dusty',
 'buttery',
 'musk',
 'potato',
 'popcorn',
 'citrus',
 'powdery',
 'onion',
 'wasabi',
 'burnt',
 'ferment

In [5]:
import torch
# Create a dictionary mapping each label to a unique index
label_to_index = {label: idx for idx, label in enumerate(all_notes)}

def multi_hot(notes):
    # Initialize a zero tensor of the appropriate size
    multi_hot_vector = torch.zeros(len(all_notes))

    # Set the corresponding positions in the tensor to 1 for each label of the item
    for label in notes:
        index = label_to_index[label]
        multi_hot_vector[index] = 1
    return multi_hot_vector

all_multihots = dict()
for d in tqdm.tqdm(dataset):
    all_multihots[(d["mol1"],d["mol2"])] = multi_hot(d["blend_notes"])
next(iter(all_multihots.items()))

100%|█████████████████████████████| 266758/266758 [00:01<00:00, 213070.27it/s]


(('CC(=CCC1C(O1)(C)C)C=C', 'CCCC(CC)O'),
 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.]))

In [6]:
import collections

notes_to_pairs = collections.defaultdict(list)
for d in tqdm.tqdm(dataset):
    notes_to_pairs[frozenset(d["blend_notes"])].append((d["mol1"],d["mol2"]))

len(notes_to_pairs), len(next(iter(notes_to_pairs.values())))

100%|████████████████████████████| 266758/266758 [00:00<00:00, 2569083.55it/s]


(6850, 4384)

In [7]:
import random

positives = dict()
for n, pairs in tqdm.tqdm(notes_to_pairs.items()):
    for p1 in pairs:
        vals = list(random.sample(pairs,MAX_COUNT)) if len(pairs) > MAX_COUNT else pairs
        positives[p1] = [p2 for p2 in vals if p1 != p2]
len(positives), next(iter(positives.items()))

100%|███████████████████████████████████| 6850/6850 [00:02<00:00, 3155.05it/s]


(266758,
 (('CC(=CCC1C(O1)(C)C)C=C', 'CCCC(CC)O'),
  [('CC(=O)OC1(CCCCC1)C#C', 'CC1(C2CCC(C2)(C1=O)C)C'),
   ('CC(=CCC1C(O1)(C)C)C=C', 'CC1=CC2C(C2(C)C)CC1C(=O)C'),
   ('CC(=CCCC(C)(C=C)OC=O)C', 'CC(C)C(=O)OC1CCC2CCCCC2C1'),
   ('C[C@]1([C@H](CCC(O1)(C)C)O)C=C', 'O=COC/C(=C\\c1ccccc1)CCCCC'),
   ('CC(C)C1=CC(=O)C(=CC=C1)O', 'CC(C)C1C(COCO1)(C)C'),
   ('CC(=O)C1=CN=CC=C1', 'CC1CC(CC(=O)O1)(C)C'),
   ('CC(CCOC(=O)C)CC(C)(C)C', 'CCCCC(=O)OCC'),
   ('CC(=CC(=O)OCCC1=CC=CC=C1)C', 'CC(CCC=C(C)C)CC(=O)OC'),
   ('CC(C)/C=C/CCCCC(=O)NCC1=CC(=C(C=C1)O)OC', 'C\\C=C(/C)\\C(=O)OCCC(C)C'),
   ('CC(C)C1=CC=C(C=C1)COC(=O)C', 'CCCCC(CO)C(=O)C'),
   ('CC(=CC(=O)OCCC1=CC=CC=C1)C', 'CCCCCO'),
   ('CC(C)(C=C)O', 'CCC1OCC2(CC(=CCC2C)C)CO1'),
   ('CC(CCOC=O)CC(C)(C)C', 'CC1(C2CCC(=C)C1C2)C'),
   ('CC(C)C/C=C(\\COC(=O)C)/C(C)C', 'CCNC(=O)C1CC(CCC1C(C)C)C'),
   ('COC1=C2C(=C(C(=C1)CC=C)OC)OCO2',
    'C[C@@H]1CC[C@H]2[C@@H]1C3[C@H](C3(C)C)CC[C@]2(C)O'),
   ('CC(C)C1C(COCO1)(C)C', 'CC1=CCC2(C1C2)C(C)C'),
   ('CC

In [8]:
missing = len([p for p, ps in positives.items() if len(ps) < MAX_COUNT])
missing, missing/len(dataset)

(32712, 0.12262799991003082)

In [9]:
mol_sets = collections.defaultdict(list)
for i,d in enumerate(dataset):
    mol_sets[d["mol1"]].append(i)
    mol_sets[d["mol2"]].append(i)

In [10]:
pairs = []
pair_to_notes = dict()
for d in dataset:
    pair = (d["mol1"],d["mol2"])
    pairs.append(pair)
    pair_to_notes[pair] = set(d["blend_notes"])

In [11]:
notes_sets = dict()
for i,d in enumerate(dataset):
    notes_sets[i] = set(d["blend_notes"])

In [12]:
# Hard negatives are anchor/negatives that share a molecule
# but do not have any notes in common.
negatives = collections.defaultdict(list)
for (mol,idcs) in tqdm.tqdm(mol_sets.items()):
    for i in idcs:
        p1 = notes_sets[i]
        for j in idcs:
            p2 = notes_sets[j]
            if bool(p1 & p2):
                continue
            negatives[pairs[i]].append(pairs[j])
len(negatives)

100%|████████████████████████████████████| 3428/3428 [00:25<00:00, 136.49it/s]


266242

In [13]:
missing = len([p for p, ns in negatives.items() if len(ns) < MAX_COUNT])
missing, missing/len(dataset)

(15554, 0.058307529671087654)

In [14]:
# # Supplement with random negatives if we have less than the required
# # number of hard negatives.
# for p, ns in tqdm.tqdm(negatives.items()):
#     notes = pair_to_notes[pair]
#     while len(ns) < MAX_COUNT:
#         other = random.choice(dataset)
#         other_pair = (other["mol1"],other["mol2"])
#         other_notes = pair_to_notes[other_pair]
#         if bool(notes & other_notes):
#             continue
#         ns.append(other_pair)
#     negatives[p] = list(random.sample(pairs,MAX_COUNT))

# missing = len([p for p, ns in negatives.items() if len(ns) != MAX_COUNT])
# len(negatives), missing, missing/len(dataset)      

In [15]:
next(iter(negatives.items()))

(('CC(=CCC1C(O1)(C)C)C=C', 'CCCC(CC)O'),
 [('CC(=CCC1C(O1)(C)C)C=C', 'CCOC(C)(CCC=C(C)C)C=C'),
  ('CC(=CCC1C(O1)(C)C)C=C', 'CC/C=C\\CCOC(=O)C1=CC=CC=C1O'),
  ('CC(=CCC1C(O1)(C)C)C=C', 'CC(C)(CCC1CCCCC1)O'),
  ('CC(=CCC1C(O1)(C)C)C=C', 'CC(=CCC[C@@](C)(C=C)O)C'),
  ('CC(=CCC1C(O1)(C)C)C=C', 'CCOC(C)OC(C)(CCC=C(C)C)C=C'),
  ('CC(=CCC1C(O1)(C)C)C=C', 'CC/C(=C/CCC(C)(C=C)OC(=O)C)/C'),
  ('CC(=CCC1C(O1)(C)C)C=C', 'CC/C(=C/CCC(C)(C=C)O)/C'),
  ('CC(=CCC1C(O1)(C)C)C=C', 'CCC(C)(CCC=C(C)C)O'),
  ('CC(=CCC1C(O1)(C)C)C=C', 'CC1CCOC(C1)CC(C)C'),
  ('CC(=CCC1C(O1)(C)C)C=C', 'CC1=CCC(=C(C)CCC=C(C)C)CC1'),
  ('CC(=CCC1C(O1)(C)C)C=C', 'CCCCCCCCOC(=O)CC1=CC=CC=C1'),
  ('CC(=CCC1C(O1)(C)C)C=C', 'CC1(CCC(C23C14C2CC(C4)C3(C)C)COC=O)C'),
  ('CC(=CCC1C(O1)(C)C)C=C', 'CCC(C)(CCCC(C)C)O'),
  ('CC(=O)OCC1CCC(C23C14C2CC(C3)C4(C)C)(C)C', 'CCCC(CC)O'),
  ('CCC(C)C=O', 'CCCC(CC)O'),
  ('CC1(CCC(C23C14C2CC(C4)C3(C)C)CO)C', 'CCCC(CC)O'),
  ('CCCC(CC)O', 'CCOC(=O)C(C)C'),
  ('CCCC(CC)O', 'C[C@]12CCCC(C1CC[C@@]([C@@H

In [16]:
all_smiles = set()
for d in dataset:
    all_smiles.add(d["mol1"])
    all_smiles.add(d["mol2"])
len(all_smiles),next(iter(all_smiles))

(3428, 'CCC=CCCC=CC=O')

In [17]:
from ogb.utils import smiles2graph
from torch_geometric.data import InMemoryDataset, download_url, Data
from torch_geometric.loader import DataLoader

def to_torch(graph):
    tensor_keys = ["edge_index", 'edge_feat', 'node_feat']
    for key in tensor_keys:
        graph[key] = torch.from_numpy(graph[key])
    return Data(x=graph["node_feat"].float(),edge_attr=graph["edge_feat"],edge_index=graph["edge_index"])

errored = 0
graph_data = dict()
for smiles in tqdm.tqdm(all_smiles):
    try:
        graph_data[smiles] = to_torch(smiles2graph(smiles))
    except AttributeError as e:
        errored += 1
errored, len(graph_data), next(iter(graph_data.items()))

  0%|                                                | 0/3428 [00:00<?, ?it/s][12:32:43] SMILES Parse Error: syntax error while parsing: InChI=1/C7H8S/c1-6-4-2-3-5-7(6)8/h2-5,8H,1H3
[12:32:43] SMILES Parse Error: Failed parsing SMILES 'InChI=1/C7H8S/c1-6-4-2-3-5-7(6)8/h2-5,8H,1H3' for input: 'InChI=1/C7H8S/c1-6-4-2-3-5-7(6)8/h2-5,8H,1H3'
 44%|███████████████▎                   | 1505/3428 [00:01<00:01, 1009.35it/s][12:32:44] SMILES Parse Error: syntax error while parsing: (C)C1=CN=CC(=N1)OC.CC(C)C1=CN=C(C=N1)OC.CC(C)C1=NC=CN=C1OC
[12:32:44] SMILES Parse Error: Failed parsing SMILES '(C)C1=CN=CC(=N1)OC.CC(C)C1=CN=C(C=N1)OC.CC(C)C1=NC=CN=C1OC' for input: '(C)C1=CN=CC(=N1)OC.CC(C)C1=CN=C(C=N1)OC.CC(C)C1=NC=CN=C1OC'
[12:32:44] Can't kekulize mol.  Unkekulized atoms: 3 4 5 6 8
100%|████████████████████████████████████| 3428/3428 [00:03<00:00, 995.67it/s]


(3,
 3425,
 ('CCC=CCCC=CC=O', Data(x=[10, 9], edge_index=[2, 18], edge_attr=[18, 3])))

In [18]:
import numpy as np
import data

pair_to_data = dict()
for d in tqdm.tqdm(dataset):
    if not d["mol1"] in graph_data or not d["mol2"] in graph_data:
        continue
    pair = (d["mol1"],d["mol2"])
    g1 = graph_data[d["mol1"]]
    g2 = graph_data[d["mol2"]]
    pair_to_data[pair] = data.combine_graphs([g1,g2])
len(pair_to_data), next(iter(pair_to_data.items()))

100%|███████████████████████████████| 266758/266758 [01:02<00:00, 4243.49it/s]


(266450,
 (('CC(=CCC1C(O1)(C)C)C=C', 'CCCC(CC)O'),
  BlendData(x=[18, 9], edge_index=[2, 34], edge_attr=[34, 3], mol_batch=[18], blend_batch=[2])))

In [19]:
valid_pairs = set(pair_to_data.keys()).intersection(set(positives.keys())).intersection(set(negatives.keys())).intersection(set(all_multihots.keys()))
len(valid_pairs), len(valid_pairs)/len(dataset)

(265934, 0.9969110579626478)

In [20]:
import json
import h5py
import base64

def encode_smiles(smiles):
    return base64.urlsafe_b64encode(smiles.encode()).decode()

with h5py.File('dataset_labels_small.h5', 'w') as f:
    for i, (pair, data) in enumerate(tqdm.tqdm(pair_to_data.items())):
        if i > 10000:
            break
        group = f.create_group(encode_smiles(json.dumps(pair)))
        group.create_dataset("pair",data=pairs[i])
        group.create_dataset("notes",data=all_multihots[pair].numpy())
        graph_group = group.create_group("graph")
        for k,v in data.items():
            graph_group.create_dataset(k,data=v.numpy())

  4%|█▏                              | 10001/266450 [00:05<02:16, 1879.35it/s]


In [21]:
import json
import h5py
import base64

def encode_smiles(smiles):
    return base64.urlsafe_b64encode(smiles.encode()).decode()

with h5py.File('dataset_labels.h5', 'w') as f:
    for i, (pair, data) in enumerate(tqdm.tqdm(pair_to_data.items())):
        group = f.create_group(encode_smiles(json.dumps(pair)))
        group.create_dataset("pair",data=pairs[i])
        group.create_dataset("notes",data=all_multihots[pair].numpy())
        graph_group = group.create_group("graph")
        for k,v in data.items():
            graph_group.create_dataset(k,data=v.numpy())

100%|███████████████████████████████| 266450/266450 [03:31<00:00, 1257.28it/s]


In [22]:
import json
import h5py
import base64

def encode_smiles(smiles):
    return base64.urlsafe_b64encode(smiles.encode()).decode()

with h5py.File('dataset_contrast.h5', 'w') as f:
    for i, (pair, data) in enumerate(tqdm.tqdm(pair_to_data.items())):
        group = f.create_group(encode_smiles(json.dumps(pair)))
        group.create_dataset("pair",data=pairs[i])
        group.create_dataset("positives",data=positives[pair])
        group.create_dataset("negatives",data=negatives[pair])
        graph_group = group.create_group("graph")
        for k,v in data.items():
            graph_group.create_dataset(k,data=v.numpy())

100%|████████████████████████████████| 266450/266450 [06:39<00:00, 666.37it/s]
